In [182]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [183]:
with open('../data/country_codes_dict.pickle', 'rb') as f:
    codes = pickle.load(f)

In [184]:
df = pd.read_csv('../data/eurovision_merged_covariates_27Jan.csv')

In [185]:
type(codes)

dict

In [186]:
# reverse the dictionary
codes = {v: k for k, v in codes.items()}

In [187]:
# variable names to lowercase and _ instead of spaces
df.columns = df.columns.str.lower().str.replace(' ', '_')
# add countries to codes in df with codes df
df['from_country_name'] = df['from_country'].map(codes)
df['country_name'] = df['country'].map(codes)

In [188]:
df

,unnamed:_0,year,from_country,votes,country,own,english,other,has_border,from_country_name,country_name
0,3,1999,AT,0.0,AT,0.0,1.0,0.0,NaN,austria,austria
1,6,1999,AT,12.0,BA,0.0,0.0,1.0,NaN,austria,bosnia and herzegovina
2,7,1999,AT,0.0,BE,0.0,1.0,0.0,NaN,austria,belgium
3,11,1999,AT,0.0,CY,1.0,0.0,0.0,NaN,austria,cyprus
4,13,1999,AT,10.0,DE,0.0,0.0,1.0,1.0,austria,germany
...,...,...,...,...,...,...,...,...,...,...,...
19214,37619,2019,SM,1.0,NO,0.0,0.0,1.0,NaN,san marino,norway
19215,37623,2019,SM,0.0,RS,0.0,0.0,1.0,NaN,san marino,serbia
19216,37624,2019,SM,12.0,RU,0.0,1.0,0.0,NaN,san marino,russia
19217,37625,2019,SM,0.0,SE,0.0,1.0,0.0,NaN,san marino,sweden


In [189]:
df2 = (df
    .groupby(['from_country', 'country'])
    .agg({'votes': 'mean'})
    .reset_index()
    .sort_values('votes', ascending=False)
)

In [190]:
# make new variable with both country codes, sorted alphabetically
df2['country_pair'] = df2[['from_country', 'country']].apply(lambda x: '_'.join(sorted(x)), axis=1)

# filter country_pairs that appear at least twice
df2 = df2[df2['country_pair'].map(df2['country_pair'].value_counts()) > 1]

# sort by country_pair
df2 = df2.sort_values('country_pair')

# add list of 1,2,1,2
# make list of len(df2) with 1,2,1,2
df2['pair'] = [1,2]*int(len(df2)/2)

In [197]:
df2

,from_country,country,votes,country_pair,pair
42,AL,AM,0.800000,AL_AM,1
87,AM,AL,0.000000,AL_AM,2
43,AL,AT,0.714286,AL_AT,1
132,AT,AL,3.666667,AL_AT,2
44,AL,AU,3.000000,AL_AU,1
...,...,...,...,...,...
2158,UA,TR,2.333333,TR_UA,2
2191,YU,TR,0.666667,TR_YU,1
2114,TR,YU,4.000000,TR_YU,2
2192,YU,UA,4.000000,UA_YU,1


In [208]:
df3.columns.values

array([('from_country', 1), ('from_country', 2), ('country', 1),
       ('country', 2), ('votes', 1), ('votes', 2), ('diff', '')],
      dtype=object)

In [207]:
#df3.columns = ['_'.join(col) for col in df3.columns.values]


TypeError: sequence item 1: expected str instance, int found

In [225]:
df3 = df2.pivot(index=['country_pair'], columns='pair', values=[ 'votes', 'from_country'])
# remove multiindex
df3.columns = df3.columns.droplevel(0)
# rename columns
df3.columns = ['votes1', 'votes2', 'from_country1', 'from_country2', ]
# difference in votes
df3['votes_diff'] = abs(df3['votes1'] - df3['votes2'])
# sort by votes_diff
df3 = df3.sort_values('votes_diff', ascending=False)
df3.head(30)

,votes1,votes2,from_country1,from_country2,votes_diff
country_pair,,,,,
AT_YU,12.0,0.0,AT,YU,12.0
CH_YU,12.0,0.0,CH,YU,12.0
FR_TR,0.214286,9.846154,TR,FR,9.631868
AZ_CZ,9.0,0.666667,CZ,AZ,8.333333
BG_MT,9.0,0.833333,MT,BG,8.166667
FR_YU,0.0,8.0,YU,FR,8.0
AM_TR,0.8,8.8,AM,TR,8.0
CZ_RU,8.0,0.0,CZ,RU,8.0
CY_YU,2.0,10.0,YU,CY,8.0


In [199]:
# pivot wider splitting on pair but keeping all other variables
df3 = df2.pivot(index='country_pair', columns='pair', values=['from_country', 'country', 'votes'])

# flatten column names
df3.columns = ['_'.join(col).strip() for col in df3.columns.values]

# calculate difference in votes
df3['diff'] = abs(df3['votes'][1] - df3['votes'][2])


In [196]:
df3['country']

pair,1,2
0,AM,AL
1,AT,AL
2,AU,AL
3,AZ,AL
4,AL,BA
...,...,...
1006,SI,UA
1007,UA,SM
1008,UA,TR
1009,TR,YU


In [162]:
# long to wide by pair
df3 = df2.pivot(index='country_pair', columns='pair', values='votes')
df3

pair,1,2
country_pair,,
AL_AM,0.800000,0.000000
AL_AT,0.714286,3.666667
AL_AU,3.000000,0.000000
AL_AZ,2.636364,1.714286
AL_BA,3.500000,5.333333
...,...,...
SI_UA,1.200000,2.266667
SM_UA,1.625000,0.000000
TR_UA,3.900000,2.333333
